# Анализ базы данных сервиса для чтения книг по подписке 

Цель: Проанализировать базу данных сервиса для чтения книг и сформулировать ценностное предложение для нового продукта.

Описание данных:

Предоставлены четыре таблицы:
- books.
- authors.
- publishers.
- ratings.
- reviews.

books- содержит данные о книгах.

Структура таблицы:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

authors- содержит данные об авторах.

Структура таблицы:

- author_id — идентификатор автора;
- author — имя автора.

publishers- содержит данные об издательствах.

- publisher_id — идентификатор издательства;
- publisher — название издательства.

ratings- содержит данные о пользовательских оценках книг.

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

reviews- содержит данные о пользовательских обзорах.

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Шаг1. Изучение файла с данными:
- Выгрузка и просмотр информации о таблицах.

Шаг 2. Исследовательский анализ:

- Рассчет количества книг, выпущенных после 1 января 2000 года;
- Рассчет количества обзоров и средней оценки для каждой книги; 
- Определение издательства, которое выпустило наибольшее число книг толще 50 страниц;
- Определиние автора с самой высокой средней оценкой книг;
- Рассчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок.

Шаг 3. Вывод по исследованию и рекомендации.

# Шаг 1. Изучение файла с данными:

Импортируем требуемые библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Выведим первые пять строк каждой таблицы:

In [2]:
books = '''
SELECT  *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
authors = '''

SELECT *
FROM authors
LIMIT 5

'''
pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
publishers = '''

SELECT *
FROM publishers
LIMIT 5

'''
pd.io.sql.read_sql(publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
ratings = '''

SELECT *
FROM ratings
LIMIT 5

'''
pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
reviews = '''

SELECT *
FROM reviews
LIMIT 5

'''
pd.io.sql.read_sql(reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Шаг 2. Исследовательский анализ

Рассчет количества книг, выпущенных после 1 января 2000 года:

In [7]:
books_count = '''
                SELECT 
                COUNT(book_id) AS amount_of_books
                FROM
                    books
                WHERE publication_date >= '2000-01-02';
             ''' 
pd.io.sql.read_sql(books_count, con = engine) 

,amount_of_books
0,819


Вывод:
- После 1 января 2000 года было выпущено 819 книг.

Рассчет количества обзоров и средней оценки для каждой книги:

In [8]:
reviews_rat = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS amount_of_reviews, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id

LEFT JOIN  ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY amount_of_reviews DESC

'''

pd.io.sql.read_sql(reviews_rat, con = engine)

,book_id,title,amount_of_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


Количество обзоров маленькое. Самая популярная книга по обзорам Сумерки. Если рассматривать рейтинг самой популярной книги, то всего 3.6. На некоторые книги вообще нет обзоров.

Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
book_publisher = '''
SELECT
     publishers.publisher AS publisher,
     COUNT(books.book_id) AS num_public
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    publisher
ORDER BY 
    num_public DESC
LIMIT 5
'''
pd.io.sql.read_sql(book_publisher, con = engine) 

,publisher,num_public
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Топ-3 издательства по количеству книг:
- Penguin Books;
- Vintage;
- Grand Central Publishing.

Самое продуктивное по выпуску книг издательство Penguin Books выпустило 42 книги. 

Определиние автора с самой высокой средней оценкой книг

In [10]:
author_rat = '''


SELECT a.author,
       AVG(r.rating) avg_rating
FROM authors AS a
JOIN books AS b ON a.author_id=b.author_id
JOIN ratings AS r ON b.book_id=r.book_id
WHERE b.book_id IN
    (SELECT b.book_id
     FROM books AS b
     JOIN ratings AS r ON b.book_id=r.book_id
     GROUP BY b.book_id
     HAVING count(r.rating_id)>=50)
GROUP BY a.author
ORDER BY avg_rating desc

LIMIT 5

'''
pd.io.sql.read_sql(author_rat, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Топ-3 автора по среднему рейтингу:
- Джоан Роулинг;
- Маркус Зусак;
- Джон Толкиен.

Автор с самым высоким средним рейтингом Роулинг. Не удивительно, кто не любит Гарри Поттера. В общей сложности тираж серии книг про Гарри Поттера по всему миру превышает 500 млн экземпляров, это одна из самых продаваемых книг в истории, что еще раз доказывает всеобщую любовь к данному писателю.

Рассчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
avg_views = '''

WITH A AS
  (SELECT username,
          count(review_id)
   FROM reviews
   WHERE username in
       (SELECT username
        FROM ratings
        GROUP BY username
        HAVING count(rating_id)>50)
   GROUP BY username)
SELECT round (SUM (COUNT) / COUNT(COUNT), 2) AS AVG
FROM A

'''

pd.io.sql.read_sql(avg_views, con = engine)


,avg
0,24.33


Среднее количество обзоров от активных пользователей 24 обзора.

# Шаг 3. Вывод по исследованию и рекомендации.

Выводы:
- Предоставлена информация о 993 книгах.
- Большая часть книг выпущена за последние 20 лет.
- Самые высокие показатели количества обзоров 7-6. Самая популярная книга по обзорам Сумерки.
- Мксимальное количество книг выпустило издательство Penguin Books(42 книги). Это около 4,2% от всех книг.
- Самый высокий рейтинг у писателия Джоан Роулинг.
- Среднее количество обзоров от активных пользователей 24.

Рекомендации:

Количество книг в приложении не такое уж большое, стоит это учесть и увеличить ассортимент.  Также очевидно, что активность пользователей в обзорах низкая, а ведь часто потенциальные покупатели, любят почитать отзывы перед тем как совершить покупку, возможно стоит улучшить интерфейс приложения, сделать более яркую и привлекательную часть приложения, где пользователь может поставить оценку и сформировать обзор.
Также можно добавить в ассортимент подкасты и аудиокниги, так как у этой нишы довольно широкая аудитория, которая увеличивается с каждым годом.